# Assignment 1
Emma McCready\
sba23001

---

### Analysis of Ireland's Transport Data




^ implement a project overview/plan like how they do in codecademy projects??

## Scoping

The below four sections are to help guide the EDA during this project and also to structure this codebook as it may get quite long. The first section defines the high-level objectives and set the intentions for the overall project. The next section is the data. Thirdly, the analysis will have to be thought through, which include the methods and questions that are aligned with the project goals. Lastly, evaluation will help us build conclusions and findings from our analysis.

### Project Goals

An aspect of Ireland's Transport data will be analysed and compared to other transport systems in other countries. The focus will be on train systems. Some questions that are posed:

- 

### Data

Dataset sources:
* link..
* link 2...

Data will be cleaned and prepared prior to analysis.

### Analysis

In this section, descriptive statistics and data visualization techniques will be employed to understand the data better. 
The datasets were not viewed before the initiation of this project and may be quite large.

### Evaluation

Lastly, it's a good idea to revisit the goals and check if the output of the analysis corresponds to the questions first set to be answered (in the goals section). This section will also reflect on what has been learned through the process, and if any of the questions were unable to be answered. This could also include limitations or if any of the analysis could have been done using different methodologies.


In [2]:
# Dependencies.. put any packages I install here:
#!pip install pandas==2.1.2
#!pip install ydata_profiling --user

In [4]:
# review this at end to ensure there's no unnecessary packages loaded

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 
from scipy.stats import norm
from scipy.stats import poisson, binom, norm
from scipy.stats import shapiro

# ML libraries
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, 
                                     StratifiedKFold, learning_curve, train_test_split)
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

<summary style="color:blue;">

# 1. Data Inspection of the Primary Dataset

This dataset was obtained from the CSO, and pertains to rail transport activity within the Republic of Ireland and rail transport activity where a journey begins or ends in the Republic of Ireland.
    
First, I need to load in the data, and then take a quick look at the data to get a sense of what I'm dealing with. I'm initially interested in what unique columns there are, the data types of the columns,

In [39]:
df1 = pd.read_csv("TCA01.csv")
df1.head(10)
#pop_data.tail()

,STATISTIC,STATISTIC Label,TLIST(A1),Year,C02617V03175,Type of Journey,UNIT,VALUE
0,TCA01,Passenger Journeys by Rail (Thousand),1981,1981,-,All journeys,Number,15374.0
1,TCA01,Passenger Journeys by Rail (Thousand),1981,1981,01,Mainline and other services,Number,7379.0
2,TCA01,Passenger Journeys by Rail (Thousand),1981,1981,02,Dublin suburban services,Number,7995.0
3,TCA01,Passenger Journeys by Rail (Thousand),1981,1981,03,DART,Number,NaN
4,TCA01,Passenger Journeys by Rail (Thousand),1981,1981,04,International journeys,Number,NaN
5,TCA01,Passenger Journeys by Rail (Thousand),1982,1982,-,All journeys,Number,12813.0
6,TCA01,Passenger Journeys by Rail (Thousand),1982,1982,01,Mainline and other services,Number,6677.0
7,TCA01,Passenger Journeys by Rail (Thousand),1982,1982,02,Dublin suburban services,Number,6136.0
8,TCA01,Passenger Journeys by Rail (Thousand),1982,1982,03,DART,Number,NaN
9,TCA01,Passenger Journeys by Rail (Thousand),1982,1982,04,International journeys,Number,NaN


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATISTIC        210 non-null    object 
 1   STATISTIC Label  210 non-null    object 
 2   TLIST(A1)        210 non-null    int64  
 3   Year             210 non-null    int64  
 4   C02617V03175     210 non-null    object 
 5   Type of Journey  210 non-null    object 
 6   UNIT             210 non-null    object 
 7   VALUE            165 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 13.3+ KB


In [10]:
#df1.describe()
df1.describe(include=object)

,STATISTIC,STATISTIC Label,C02617V03175,Type of Journey,UNIT
count,210,210,210,210,210
unique,1,1,5,5,1
top,TCA01,Passenger Journeys by Rail (Thousand),-,All journeys,Number
freq,210,210,42,42,210


In [8]:
#checking amount of missing data..
df1.isnull().sum()

STATISTIC           0
STATISTIC Label     0
TLIST(A1)           0
Year                0
C02617V03175        0
Type of Journey     0
UNIT                0
VALUE              45
dtype: int64

In [11]:
duplicates = df1.duplicated()
print(duplicates.value_counts())

False    210
dtype: int64


In [17]:
df1["TLIST(A1)"].equals(df1["Year"])

True

### Observations on the above:

**From calling `.head()`:**
* STATISTIC and STATISTIC Label columns redundant. UNIT also irrelevant. VALUE column refers to the number of passender journeys by rail in thousands.
* Not sure what TLIST(A1) is, possibly an error from when the data was downloaded from the CSO website? Will have to check if the columns are equal and if so drop one of the columns.
* Similarly, not sure what C02617V03175 refers to - will have to check values. From looking at .head(10), it might just be an index key for the Type of Journey values?


**From calling `.info()`:**
* Dataset contains 210 observations
* The statistical information extracted (ie. the mean, std, etc.) isn't reliable at this early stage, as there's a total variable also ('All journeys')


**From calling `.describe()`:**
* For the numerical features:
    * Year column: ranges from 1981 to 2022.
    * Missing values in VALUE column.. will need to investigate and decide what to do with them.
* For the categorical features:
    * "STATISTIC Label" and "UNIT" only have one unique value each ("Passenger Journeys by Rail (Thousand)" and "number" respectively)
    * "C02617V03175" and "Type of Journey" both have 5 unique values - this reinforces my assumption that C02617V03175 is a sort of key for the latter feature. This will need to be confirmed.


**From calling `.isnull().sum()`:**
* 45 missing values in the VALUE field. Will need to investigate and figure out why these are missing. 45 is significant as dataset contains only 210 observations - this means 21.4% of the observations contain missing values.


**From calling `duplicates.value_counts()`:**
* There are no duplicated observations.

**From calling `df1["TLIST(A1)"].equals(df1["Year"])`:**
* Output is true, implying these columns are the exact same, and one of these columns can be dropped.



One of the labels under the "Type of Journey" feature is "International journeys". Noting that Ireland is an island with no linking tunnels, I'd like to get a closer look at this. In doing this, I'll confirm that "C02617V03175" is just an index key (and can therefore be dropped to tidy the dataset), and filter both "Type of Journey" and "C02617V03175" for the relevant variable:

In [31]:
int_journeys1 = df1[df1["Type of Journey"] == "International journeys"]
int_journeys2 = df1[df1["C02617V03175"] == "04"]
#int_journeys1.info()
#int_journeys2.info()
int_journeys1.equals(int_journeys2)

True

In [37]:
int_journeys1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 4 to 209
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATISTIC        42 non-null     object 
 1   Year             42 non-null     int64  
 2   C02617V03175     42 non-null     object 
 3   Type of Journey  42 non-null     object 
 4   VALUE            21 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.0+ KB


This confirms that "C02617V03175" is an index key, and can therefore be dropped. 
From calling .info(), I can see that there are 21 non-null values, and therefore some international journeys taking place. From my own knowledge of the Irish Rail system, I can assume that this is the Dublin -> Belfast route, but there's no information in the current dataset which can confirm this.

In [33]:
int_journeys1.isnull().sum()

STATISTIC           0
Year                0
C02617V03175        0
Type of Journey     0
VALUE              21
dtype: int64

<summary style="color:blue;">

# 2. Data Tidying


<summary style="color:blue;">

###### 2.1 Dropping "Statistic Label", "TLIST(A1)", "C02617V03175" and "UNIT" columns 
    
</summary>


I was planning on removing the "Statistic Label" as I felt it's providing no purpose in the analysis. However, I redecided to keep it as it contains important information which may be forgotten in the future (that the VALUEs for each observation are in thousands)
    
    
I'm also removing the "UNIT" column, as I would rather include this in the "VALUE" column (by multiplying the values by a thousand)

As noted above, "TLIST(A1)" is a duplication of the "Year" column, and so should be dropped. Further, "C02617V03175" can be dropped as it is just an index key, and I'd rather work with the strings in the "Type of Journey" column which are a full description of the type of journey. 
    
    
Doing this should make the dataframe less clunky and easier to explore

However, I will keep the "Statistic" column as this could be a useful reference when I merge data to this dataset in the future.

In [40]:
df1 = df1.drop(columns = ['TLIST(A1)', 'C02617V03175', 'UNIT'])
#df1.head()

<summary style="color:blue;">

###### 2.2 Renaming the column headings
    
</summary>
    
This is just to make it easier to write code, and also to make headings reflect the data their respective column contains more accurately. I also want to make them all lowercase so it's more convenient to type. I won't touch the STATISTIC column heading as it's unlikely I'll be utilising it.

In [43]:
df1.head()

,STATISTIC,year,journey_type,passenger_count
0,TCA01,1981,All journeys,15374.0
1,TCA01,1981,Mainline and other services,7379.0
2,TCA01,1981,Dublin suburban services,7995.0
3,TCA01,1981,DART,NaN
4,TCA01,1981,International journeys,NaN


In [42]:
df1 = df1.rename(columns = {'Year':'year', 
                            'Type of Journey':'journey_type',
                            'VALUE':'passenger_count'})
df1.columns

Index(['STATISTIC', 'year', 'journey_type', 'passenger_count'], dtype='object')

<summary style="color:blue;">

###### 2.3 Missing Values
    
</summary>

First, I want to look at the missing values, so I'll move them into a new dataset. I'm particularly interested in what values are actually missing. Also interested in what year the missing data is, so I can decide if limiting the analysis to a particular year range is better. As noted from calling .info() on the dataset earlier, only "passenger_count" contains missing values, so I can streamline the code a little:
    

In [56]:
missing_count = df1['passenger_count'].isnull().sum()

print('There are', missing_count, 'missing values in the total_migration column. The total percentage of missing values is', round((missing_count/len(df1) * 100), 2), '%')

missing = pd.isnull(df1['passenger_count'])
missing_data = df1[missing]
#missing_data

There are 45 missing values in the total_migration column. The total percentage of missing values is 21.43 %


## Summary of the data tidying:

I dropped the irrelevant columns but kept "Statistic label" to remember that `passenger_count` is in thousands

